<a href="https://colab.research.google.com/github/bitaller/walmart/blob/master/walmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Really need these
import pandas as pd 
import numpy as np
from numpy import *


#Handy for debugging
import gc
import time
import warnings
import os

#Date stuff
from datetime import datetime
from datetime import timedelta

#Do some statistics
from scipy.misc import imread
from scipy import sparse
import scipy.stats as ss
import math

#Nice graphing tools
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls

#Machine learning tools
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy import sparse

## Keras for deep learning
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.models import Sequential
from keras import regularizers
from keras import optimizers

## Performance measures
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [15]:
ls walmart/dataset

features.csv  stores.csv  test.csv  train.csv


In [0]:
#Merge info
def mergeData(df):
    features =pd.read_csv('walmart/dataset/features.csv')
    storesdata =pd.read_csv('walmart/dataset/stores.csv')
    df = pd.merge(df, features, on=['Store','Date','IsHoliday'],
                  how='inner')
    df = pd.merge(df, storesdata, on=['Store'],
                  how='inner')
    return df

#http://scikit-learn.org/stable/auto_examples/plot_cv_predict.html
def plot_prediction(predicted,true,desciption):
    fig, ax = plt.subplots()
    ax.scatter(true, predicted, edgecolors=(0, 0, 0))
    ax.plot([true.min(), true.max()], [true.min(), true.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted by '+desciption)
    ax.plot([-30,30], [0,0], 'k-')   
    ax.plot([0,0], [-30,30], 'k-')
    plt.show()
def binary(movement):
    """
    Converts percent change to a binary 1 or 0, where 1 is an increase and 0 is a decrease/no change
    
    """
    #Empty arrays where a 1 represents an increase in price and a 0 represents a decrease in price
    direction = np.empty(movement.shape[0])
    #If the change in price is greater than zero, store it as a 1
    #If the change in price is less than zero, store it as a 0
    for i in range(movement.shape[0]):
        if movement[i] > 0:
            direction[i] = 1
        else:
            direction[i]= 0
    return direction

def scatterplots(feature, label):
    x = feature
    y = df['Weekly_Sales']
    plt.scatter(x, y)
    plt.ylabel('sales')
    plt.xlabel(label)
    plt.show()

In [17]:
print('Reading data...')
print(os.listdir('walmart/dataset/'))

Reading data...
['stores.csv', 'features.csv', 'test.csv', 'train.csv']


In [26]:
dataSource = 1
if dataSource==1:
    train = mergeData(pd.read_csv('./walmart/dataset/train.csv'))
    test = mergeData(pd.read_csv('./walmart/dataset/test.csv'))
    train['Split'] = 'Train'
    test['Split'] = 'Test'
    print(test.head())
else: 
    train = pd.read_csv('../input/course-material-walmart-challenge/train.csv')
    test = pd.read_csv('../input/course-material-walmart-challenge/test.csv')
    train['Split'] = 'Train'
    test['Split'] = 'Test'
    test.head()

   Store  Dept        Date  IsHoliday  Temperature  Fuel_Price  MarkDown1  \
0      1     1  2012-11-02      False        55.32       3.386    6766.44   
1      1     2  2012-11-02      False        55.32       3.386    6766.44   
2      1     3  2012-11-02      False        55.32       3.386    6766.44   
3      1     4  2012-11-02      False        55.32       3.386    6766.44   
4      1     5  2012-11-02      False        55.32       3.386    6766.44   

   MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment Type  \
0     5147.7      50.82     3639.9    2737.42  223.462779         6.573    A   
1     5147.7      50.82     3639.9    2737.42  223.462779         6.573    A   
2     5147.7      50.82     3639.9    2737.42  223.462779         6.573    A   
3     5147.7      50.82     3639.9    2737.42  223.462779         6.573    A   
4     5147.7      50.82     3639.9    2737.42  223.462779         6.573    A   

     Size Split  
0  151315  Test  
1  151315  Test  
2 

In [23]:
test.head()

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Split
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,A,151315,Test
1,1,2,2012-11-02,False,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,A,151315,Test
2,1,3,2012-11-02,False,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,A,151315,Test
3,1,4,2012-11-02,False,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,A,151315,Test
4,1,5,2012-11-02,False,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,A,151315,Test


In [12]:
!git clone https://github.com/bitaller/walmart.git

Cloning into 'walmart'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [0]:
rm -rf walmart

In [13]:
ls walmart

dataset/  Readme.txt


In [14]:
ls walmart/dataset

features.csv  stores.csv  test.csv  train.csv
